In [1]:
import numpy as np
from numba import njit
from random import randint
import time
import timeit

In [2]:
np.binary_repr(np.int16(-4), width=16)

'1111111111111100'

In [3]:
@njit
def run():
    d = np.zeros((64,64), dtype=np.uint8) #only 1 or 0
    U = np.zeros((64,64), dtype=np.int16) #can be negative
    V = np.zeros((64,64), dtype=np.uint8) #only 1 or 0
    dU = np.zeros((64,64), dtype=np.int8)#can be negative
    sums = np.zeros(64,dtype=np.uint8)
    mask1 = np.int16(-4)
    mask2 = np.int16(32764)
    for i in range(64): #initialize
        for j in range(i+1,64):
            dx = (i//8)-(j//8)
            dy = (i%8)-(j%8)
            if(((dx*dx)+(dy*dy))==5):
                d[i][j] = 1
                d[j][i] = 1
                V[i][j] = randint(0,1)
                V[j][i] = V[i][j]
    
    for counter in range(200000):
        
#         np.sum(V,1,np.int8,sums)
        for i in range(64):
            sums[i] = 0
            for j in range(64):
                sums[i] += V[i][j]
        
#                 if d[i][j] == 1:
#                     dU[i][j] = 4 - (sums[i] + sums[j])
        
#         np.add(U,dU,U)
        for i in range(64):
            for j in range(64):
                if d[i][j] == 1:
                    U[i][j] += (4 - (sums[i] + sums[j]))
                    V[i][j] = (U[i][j]>3) + (U[i][j]&mask1 == 0)*(V[i][j]) #fastest way I think
                    #U[i][j] += dU[i][j]
#                     if U[i][j] > 3:
#                         V[i][j] = 1
#                     elif U[i][j] < 0:
#                         V[i][j] = 0
#                     else:
#                         pass
        
        if counter % 1000 == 0:
            for i in range(64):
                for j in range(64):
                    dU[i][j] = d[i][j] * (4-(sums[i]+sums[j]))
            sum_dU = np.sum(np.absolute(dU))
            max_U = np.max(np.absolute(U))
            if max_U > 255: 
                print("diverge")
                return [(np.int64(x), np.int64(x)) for x in range(0)]            
            if sum_dU == 0:
                soln = [(np.int64(x), np.int64(x)) for x in range(0)]
                for i in range(64):
                    for j in range(64):
                        if V[i][j] == 1:
                            soln.append((i,j))
#                             print(i, j)
                return soln
    print("incomplete")
    return [(np.int64(x), np.int64(x)) for x in range(0)]
    

In [4]:
def get_sol(file_f=None):
    while True:
        soln = run()
        if len(soln) > 0:
            edges = [(min(edge),max(edge)) for edge in soln]
            edges = sorted(list(set(edges)), key=lambda x: x[0])
#             print(len(edges))
            sol = np.zeros(64, dtype=np.int64)
            sol[0]=1
            cr=0
            for q in range(1,64):
                for l in range(len(edges)):
                    if edges[l][0] == cr or edges[l][1] == cr:
                        oth = edges[l][0] if edges[l][1] == cr else edges[l][1]
                    if sol[oth]==0:
                        sol[oth]=q+1;
                        cr=oth;
                        break
            if 0 in sol:
                continue

            for i in range(8):
                for j in range(8):
                    if file_f is not None:
                        file_f.write("{} ".format(str(sol[i*8+j]).ljust(2)))
                    else:
                        print(str(sol[i*8+j]).ljust(2), end=' ')
                if file_f is not None:
                    file_f.write("\n")
                else:
                    print()
            break

In [9]:
get_sol()

incomplete
incomplete
incomplete
incomplete
incomplete
incomplete
incomplete
1  54 3  42 63 40 25 22 
4  43 64 53 24 21 62 39 
55 2  17 20 41 38 23 26 
18 5  44 37 52 15 46 61 
35 56 19 16 45 60 27 14 
6  9  36 51 12 49 30 47 
57 34 11 8  59 32 13 28 
10 7  58 33 50 29 48 31 


In [5]:
times = []
for i in range (1000):
#     f = open("outputs_simd_1000_loop.txt","a")
    start = time.time()
#     get_sol(f)
    get_sol()
    taken = int(time.time()-start)
#     f.write("{} seconds taken.\n".format(str(taken)))
    print("{} seconds taken.".format(str(taken)))
    times.append(taken)
#     print(i)
#     f.close()

1  22 49 34 31 24 53 44 
50 35 64 23 52 45 32 25 
21 2  51 48 33 30 43 54 
36 63 4  29 46 55 26 13 
3  20 47 56 27 14 9  42 
62 37 28 5  8  57 12 15 
19 6  39 60 17 10 41 58 
38 61 18 7  40 59 16 11 
31 seconds taken.
1  60 29 54 33 40 31 42 
58 53 64 3  30 43 34 39 
61 2  59 28 55 32 41 44 
52 57 4  63 16 11 38 35 
5  62 25 56 27 36 45 12 
24 51 22 15 48 17 10 37 
21 6  49 26 19 8  13 46 
50 23 20 7  14 47 18 9  
57 seconds taken.
1  30 11 24 45 28 61 22 
12 41 64 29 10 23 46 27 
31 2  33 44 25 62 21 60 
42 13 40 63 20 9  26 47 
3  32 43 34 55 48 59 8  
14 39 16 19 6  35 54 51 
17 4  37 56 49 52 7  58 
38 15 18 5  36 57 50 53 
40 seconds taken.
1  30 59 22 63 32 57 54 
60 21 64 31 58 55 24 33 
29 2  11 62 23 26 53 56 
20 61 46 27 10 51 34 25 
3  28 19 12 35 40 9  52 
18 45 16 47 6  37 50 39 
15 4  43 36 13 48 41 8  
44 17 14 5  42 7  38 49 
10 seconds taken.
1  52 21 56 63 58 5  18 
22 55 64 15 4  19 40 59 
51 2  53 20 57 62 17 6  
54 23 14 3  16 41 60 39 
13 50 25 46 61 32 7  42 
24 

KeyboardInterrupt: 

In [ ]:
t = np.array(sorted(times))
print("average: {}".format(np.average(t)))
print("std dev: {}".format(np.std(t)))
print("median: {}".format(np.median(t)))
print("75th percentile: {}".format(t[74]))
i = 0
while t[i] <= 60 and i < len(t):
    i+=1
print("{} percent less than 1 min".format(((i+1)*100)/len(t)))